# 統計モデリング概論 DSHC 2022

岩嵜 航 (Watal M. Iwasaki, PhD)<br>
東北大学 生命科学研究科 進化ゲノミクス分野 特任助教

2022-08-17 東京海上 Data Science Hill Climb<br>
https://heavywatal.github.io/slides/tokiomarine2022/

## 環境セットアップ

In [ ]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import special

rng = np.random.default_rng(seed=24601)
sys.version

## とにかくGLMを使ってみる練習
### OLS (復習)
まず、OLSによる直線当てはめの復習。

In [ ]:
sample_size = 300
true_intercept = -3
true_coef = 3
x = rng.uniform(0.4, 1.7, sample_size)
lambda_ = np.exp(true_intercept + true_coef * x)
y = rng.poisson(lambda_)
df = pd.DataFrame(dict(x=x, y=y))
print(df)

In [ ]:
model = smf.ols("y ~ x", df)
result = model.fit()
df_pred = df.assign(pred=lambda _: result.predict(_))
grid = sns.FacetGrid(df_pred)
grid.map(sns.scatterplot, "x", "y")
grid.map(sns.lineplot, "x", "pred")

### GLMで直線回帰
`glm()` を使う以外の操作は共通。

In [ ]:
model = smf.glm("y ~ x", df)
result = model.fit()
df_pred = df.assign(pred=lambda _: result.predict(_))
grid = sns.FacetGrid(df_pred)
grid.map(sns.scatterplot, "x", "y")
grid.map(sns.lineplot, "x", "pred")

In [ ]:
print(result.params)

デフォルトでは正規分布・恒等リンクなのでOLSと同じ結果になった。

次に、確率分布とリンク関数を変えてみよう。
大本命、ポアソン分布・指数リンクを試す。

### ポアソン回帰
`family` の指定以外はさっきと全く同じ。
`Poinson()` の `link` はデフォルトで `Log()` なので省略可能。

In [ ]:
poisson = sm.families.Poisson(link=sm.families.links.Log())
model = smf.glm("y ~ x", df, family=poisson)
result = model.fit()
df_pred = df.assign(pred=lambda _: result.predict(_))
grid = sns.FacetGrid(df_pred)
grid.map(sns.scatterplot, "x", "y")
grid.map(sns.lineplot, "x", "pred")

In [ ]:
print(result.params)

いい感じにできた。

### 重回帰: 複数の説明変数を同時に扱う
ビールの注文数が気温と湿度の両方に依存して増加するデータを作る。

In [ ]:
sample_size = 200
true_intercept = 3
true_coefs = {"temperature": 0.05, "humidity": 0.006}
temperature = rng.uniform(8, 32, sample_size)
humidity = rng.uniform(20, 80, sample_size)
lambda_ = np.exp(
    true_intercept
    + true_coefs["temperature"] * temperature
    + true_coefs["humidity"] * humidity
)
beer_sales = rng.poisson(lambda_)
_dic = {
    "temperature": temperature,
    "humidity": humidity,
    "beer_sales": beer_sales,
}
df = pd.DataFrame(_dic)
print(df)

In [ ]:
fig, ax = plt.subplots(ncols=2)
sns.scatterplot(x="temperature", y="beer_sales", hue="humidity", data=df, ax=ax[0])
sns.scatterplot(x="humidity", y="beer_sales", hue="temperature", data=df, ax=ax[1])

縦軸は上限が無さそうなカウントデータで、x軸に対して指数的な増加。

In [ ]:
poisson = sm.families.Poisson(link=sm.families.links.Log())
model = smf.glm("beer_sales ~ temperature + humidity", df, family=poisson)
result = model.fit()
print(result.params)

In [ ]:
from itertools import product  # noqa: E402

it = product(range(8, 33, 4), range(20, 90, 10))
df_pred = pd.DataFrame(list(it), columns=["temperature", "humidity"])
df_pred = df_pred.assign(pred=lambda _: result.predict(_))
fig, ax = plt.subplots(ncols=2)
sns.scatterplot(x="temperature", y="beer_sales", hue="humidity", data=df, ax=ax[0])
sns.lineplot(x="temperature", y="pred", hue="humidity", data=df_pred, ax=ax[0])
sns.scatterplot(x="humidity", y="beer_sales", hue="temperature", data=df, ax=ax[1])
sns.lineplot(x="humidity", y="pred", hue="temperature", data=df_pred, ax=ax[1])

### ロジスティック回帰
客10人中y人がビールを注文した。
その日の気温xによって割合が変化した。

In [ ]:
# Parameters
n_traials = 10
true_intercept = -3
true_coef = 0.3
sample_size = 200
# Generate random numbers
temperature = rng.uniform(-10, 35, sample_size)
logit_p = true_intercept + true_coef * temperature
p = special.expit(logit_p)
beer_sales = rng.binomial(n_traials, p, sample_size)
_dic = {
    "temperature": temperature,
    "beer_sales": beer_sales,
    "failures": n_traials - beer_sales,
}
df = pd.DataFrame(_dic)
print(df)

In [ ]:
grid = sns.FacetGrid(df)
grid.map(sns.scatterplot, "temperature", "beer_sales")

上限10のシグモイド型の曲線でつながりそう。誤差は二項分布。

縦軸・応答変数の設定が少し特殊。
単にk回成功ではなく **n回のうちk回** 成功したという情報を使うため、
成功数と失敗数の両方を左辺に置く必要がある。

In [ ]:
binom = sm.families.Binomial(link=sm.families.links.Logit())
model = smf.glm("beer_sales + failures ~ temperature", df, family=binom)
result = model.fit()
print(result.params)

`predict()` は割合を返してくるので、
作図するときは試行数をかけるか縦軸を割合にする。

In [ ]:
df_pred = df.assign(pred=lambda _: n_traials * result.predict(_))
grid = sns.FacetGrid(df_pred)
grid.map(sns.scatterplot, "temperature", "beer_sales")
grid.map(sns.lineplot, "temperature", "pred")

### 分散分析: GLM with 質的(カテゴリカル)変数


In [ ]:
sample_size = 200
true_intercept = 70
true_coefs = {"temp": 3, "sunny": 20, "rainy": -20}
sd = 10
weather_levels = ["cloudy", "sunny", "rainy"]

In [ ]:
weather = rng.choice(weather_levels, sample_size, replace=True)
_dic = {
    "temperature": rng.uniform(8, 32, sample_size),
    "weather": pd.Categorical(weather, categories=weather_levels),
}
_df = pd.DataFrame(_dic)

df = (
    _df.join(pd.get_dummies(_df["weather"]))
    .drop("cloudy", axis=1)
    .assign(
        mu=lambda _: true_intercept
        + true_coefs["temp"] * _["temperature"]
        + true_coefs["sunny"] * _["sunny"]
        + true_coefs["rainy"] * _["rainy"]
    )
    .assign(beer_sales=lambda _: rng.normal(_["mu"], sd))
)
print(df)

In [ ]:
grid = sns.FacetGrid(df, hue="weather")
grid.map(sns.scatterplot, "weather", "beer_sales", alpha=0.6)

In [ ]:
gaussian = sm.families.Gaussian()
model = smf.glm("beer_sales ~ weather", df, family=gaussian)
result = model.fit()
print(result.params)

In [ ]:
df_pred = df.assign(pred=lambda _: result.predict(_))
grid = sns.FacetGrid(df_pred, hue="weather")
grid.map(sns.scatterplot, "weather", "beer_sales", alpha=0.6)
grid.map(sns.scatterplot, "weather", "pred", color="black", marker="x", s=120)

### 共分散分析: GLM with 質的変数 + 量的変数


In [ ]:
grid = sns.FacetGrid(df, hue="weather")
grid.map(sns.scatterplot, "temperature", "beer_sales", alpha=0.6)
grid.add_legend()

In [ ]:
gaussian = sm.families.Gaussian()
model = smf.glm("beer_sales ~ weather + temperature", df, family=gaussian)
result = model.fit()
print(result.params)

In [ ]:
df_pred = df.assign(pred=lambda _: result.predict(_))
grid = sns.FacetGrid(df_pred, hue="weather")
grid.map(sns.scatterplot, "temperature", "beer_sales", alpha=0.6)
grid.map(sns.lineplot, "temperature", "pred")
grid.add_legend()

### 交互作用
ビール売上の温度依存性が天気によって異なる。

In [ ]:
sample_size = 200
sd = 10
true_intercept = 100
true_coefs = {"sunny": -30, "temp": 3, "sunny:temp": 2}
weather_levels = ["rainy", "sunny"]
temperature = rng.uniform(8, 32, sample_size)
weather = rng.choice(weather_levels, sample_size, True)
_dic = {
    "temperature": rng.uniform(8, 32, sample_size),
    "weather": pd.Categorical(weather, categories=weather_levels),
}
_df = pd.DataFrame(_dic)
df = (
    _df.join(pd.get_dummies(_df["weather"]))
    .assign(
        mu=lambda _: true_intercept
        + true_coefs["temp"] * _["temperature"]
        + true_coefs["sunny"] * _["sunny"]
        + true_coefs["sunny:temp"] * _["sunny"] * _["temperature"]
    )
    .assign(beer_sales=lambda _: rng.normal(_["mu"], sd))
)
print(df)

In [ ]:
grid = sns.FacetGrid(df, hue="weather")
grid.map(sns.scatterplot, "temperature", "beer_sales", alpha=0.6)
grid.add_legend()

In [ ]:
gaussian = sm.families.Gaussian()
formula = "beer_sales ~ weather + temperature + weather:temperature"
model = smf.glm(formula, df, family=gaussian)
result = model.fit()
print(result.params)

In [ ]:
df_pred = df.assign(pred=lambda _: result.predict(_))
grid = sns.FacetGrid(df_pred, hue="weather")
grid.map(sns.scatterplot, "temperature", "beer_sales", alpha=0.6)
grid.map(sns.lineplot, "temperature", "pred")
grid.add_legend()

ほかに利用可能な確率分布・リンク関数などはstatsmodels公式サイトを参照:
<https://www.statsmodels.org/stable/glm.html>

一旦ここまで。講義スライドに戻る。

----

## penguins データで単回帰と重回帰の練習
まずデータをダウンロード。

In [ ]:
penguins = sm.datasets.get_rdataset("penguins", "palmerpenguins", True).data
print(penguins)

### 単回帰の練習

Step 1. まず作図

どうやら、重いペンギンほど翼長も長い。

In [ ]:
grid = sns.FacetGrid(penguins)
grid.map(sns.scatterplot, "body_mass_g", "flipper_length_mm")

Step 2. モデル作成、フィッティング

とりあえず正規分布・恒等リンクで。

In [ ]:
formula = "flipper_length_mm ~ body_mass_g"
model1 = smf.glm(formula, data=penguins)
results1 = model1.fit()
print(results1.params)

In [ ]:
print(results1.llf)

In [ ]:
print(results1.aic)

Step 3. フィッティング結果を作図

In [ ]:
pen_pred = penguins.assign(pred=results1.predict(penguins))
grid = sns.FacetGrid(pen_pred)
grid.map(sns.scatterplot, "body_mass_g", "flipper_length_mm")
grid.map(sns.lineplot, "body_mass_g", "pred")

### 重回帰の練習

Step 1. まず作図

種によって色分けしてみると、傾向の違いが見える。

In [ ]:
palette = {"Adelie": "#ff6600", "Gentoo": "#c35bcc", "Chinstrap": "#007174"}
grid = sns.FacetGrid(pen_pred, hue="species", palette=palette)
grid.map(sns.scatterplot, "body_mass_g", "flipper_length_mm")
grid.add_legend()

Step 2. モデル作成、フィッティング

Adelieを基準に、ChinstrapとGentooはそれより長め。<br>
体重の効果は単回帰のときより小さい。

In [ ]:
formula = "flipper_length_mm ~ body_mass_g + species"
model2 = smf.glm(formula, data=penguins)
results2 = model2.fit()
print(results2.params)

In [ ]:
print(results2.llf)

In [ ]:
print(results2.aic)

Step 3. フィッティング結果を作図

In [ ]:
pen_pred = penguins.assign(pred=results2.predict(penguins))
grid = sns.FacetGrid(pen_pred, hue="species", palette=palette)
grid.map(sns.scatterplot, "body_mass_g", "flipper_length_mm")
grid.map(sns.lineplot, "body_mass_g", "pred")
grid.add_legend()

**傾き**も種によって違うかも。**交互作用**を入れてみたい。

### 重回帰+交互作用の練習

Step 2. モデル作成、フィッティング

Adelieを基準に、Chinstrapの傾きが結構違う。<br>
切片の違いは解釈しにくくなった。

In [ ]:
formula = "flipper_length_mm ~ body_mass_g + species + body_mass_g:species"
model3 = smf.glm(formula, data=penguins)
results3 = model3.fit()
print(results3.params)

In [ ]:
print(results3.llf)

In [ ]:
print(results3.aic)

Step 3. フィッティング結果を作図

In [ ]:
pen_pred = penguins.assign(pred=results3.predict(penguins))
grid = sns.FacetGrid(pen_pred, hue="species", palette=palette)
grid.map(sns.scatterplot, "body_mass_g", "flipper_length_mm")
grid.map(sns.lineplot, "body_mass_g", "pred")
grid.add_legend()

----

## GLMの練習

🔰クチバシの長さと深さで同じ解析をやってみよう。

In [ ]:
sns.lmplot(
    x="bill_length_mm", y="bill_depth_mm", hue="species", palette=palette, data=penguins
)

In [ ]:

# pyright: reportGeneralTypeIssues=false
# pyright: reportMissingParameterType=false
# pyright: reportMissingTypeStubs=false
# pyright: reportUnknownArgumentType=false
# pyright: reportUnknownLambdaType=false
# pyright: reportUnknownMemberType=false
# pyright: reportUnknownParameterType=false
# pyright: reportUnknownVariableType=false